In [1]:
# ddqn = DDQN(latent_dim, action_dim).to(device)
# optimizer = torch.optim.Adam(ddqn.parameters(), lr=learning_rate)

# for epoch in range(num_epochs):
#     for (state, action, reward, next_state, done) in replay_buffer:
#         state = vae.encoder(state).detach()
#         next_state = vae.encoder(next_state).detach()
#         q_values = ddqn(state)
#         next_q_values = ddqn(next_state)
#         target = reward + gamma * torch.max(next_q_values, dim=1)[0] * (1 - done)
#         loss = F.mse_loss(q_values.gather(1, action.unsqueeze(1)), target.unsqueeze(1))
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

import torch
import numpy as np
from torch import nn
import random
import torch.nn.functional as F
import collections
from torch.optim.lr_scheduler import StepLR
from CommunicationP3DX import CommunicationP3DX
from Agent import AgentClass
from std_srvs.srv import Empty
import rospy
import time
import shelve
import os
from datetime import datetime
from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns

workspace_folder_path ='/media/xnd/7A309A87309A49D1/sia_23/25k/'

agg_method = 'mean'
n_sectors = 6

workspace = shelve.open(workspace_folder_path+'/wsh_{metodo}{n_setores}completo.out'.format(
        metodo = agg_method,n_setores = n_sectors))
# true if the key exists
vars = list(workspace.keys())
print(workspace)
print(vars)
hist_dict = workspace['hist_dict']

['Agent', 'AgentClass', 'CommunicationP3DX', 'Empty', 'Memory', 'QNetwork', 'QNetworkCNN', 'Q_1', 'StepLR', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '__warningregistry__', 'actions_angular', 'actions_linear', 'datetime', 'device', 'evaluate', 'filename', 'goal_zones_x', 'goal_zones_y', 'hist_dict', 'key', 'laser_scan_state_type_atual', 'main', 'my_shelf', 'n_sectors', 'path', 'pause_physics_client', 'performance', 'reset_simulation', 'select_action', 'str_hora_agr', 'str_hora_inicio_treino', 'theta_atual', 'train', 'unpause_physics_client', 'update_parameters']


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

# class VAE(nn.Module):
#     def __init__(self, input_dim, latent_dim):
#         super(VAE, self).__init__()
#         self.encoder = nn.Sequential(
#             nn.Linear(input_dim, 500),
#             nn.ReLU(),
#             nn.Linear(500, 128),
#             nn.ReLU(),
#             nn.Linear(128, latent_dim)
#         )
#         self.decoder = nn.Sequential(
#             nn.Linear(latent_dim, 500),
#             nn.ReLU(),
#             nn.Linear(500, 128),
#             nn.ReLU(),
#             nn.Linear(128, input_dim),
#             nn.Sigmoid()
#         )

#     def forward(self, x):
#         z = self.encoder(x)
#         return self.decoder(z), z

In [3]:
# creating dummy targets (float values)
# episode = 9900

random_episodes =np.random.randint(2000,20000,15000)
for random_ep_idx, episode in enumerate(random_episodes):
    if random_ep_idx%100==0: print('random_ep_idx = {ep_id}'.format(ep_id =random_ep_idx))
    # shape = (727, 16)
    trasnposed_scans = np.asarray([np.asarray(scan_i) for scan_i in hist_dict['scan'][episode]]).transpose()    

    if random_ep_idx==0:
        df_episode_positions = pd.DataFrame({#'i':range(len(hist_dict['pos'][episode])),
                                            'x':[pos[0] for pos in hist_dict['pos'][episode]], 
                                            'y':[pos[1] for pos in hist_dict['pos'][episode]]})

        for scan_idx in range(0,727):
            df_episode_positions['scan_'+str(scan_idx)] = pd.to_numeric([float(scan_i)  for scan_i in trasnposed_scans[scan_idx]])

    else:
        new_episode_positions = pd.DataFrame({'x':[pos[0] for pos in hist_dict['pos'][episode]], 
                                            'y':[pos[1] for pos in hist_dict['pos'][episode]]})
        
        for scan_idx in range(0,727):
            new_episode_positions['scan_'+str(scan_idx)] = pd.to_numeric([float(scan_i)  for scan_i in trasnposed_scans[scan_idx]])

        df_episode_positions = pd.concat([df_episode_positions,new_episode_positions])
        

    df_episode_positions = df_episode_positions.replace(np.Inf, np.NaN).fillna(5)

df_episode_positions.to_pickle('dataset_large_2_vae.pickle')

random_ep_idx = 0
random_ep_idx = 100
random_ep_idx = 200
random_ep_idx = 300
random_ep_idx = 400
random_ep_idx = 500
random_ep_idx = 600


KeyboardInterrupt: 

In [3]:
df_episode_positions = pd.read_pickle('dataset_large_vae.pickle')

In [20]:
# creating tensor from targets_df 
data = df_episode_positions.values # .to_numpy()
torch_tensor = torch.tensor(data)

# printing out result
print(torch_tensor)

tensor([[-3.5975,  1.8683,  5.0000,  ...,  2.1260,  2.2045,  2.2320],
        [-3.0583,  1.6770,  1.5989,  ...,  5.0000,  5.0000,  5.0000],
        [-2.6804,  2.1618,  3.0064,  ...,  5.0000,  5.0000,  5.0000],
        ...,
        [-8.4203,  1.2900,  5.0000,  ...,  1.1412,  1.1663,  1.1652],
        [-9.0506,  1.2788,  5.0000,  ...,  2.4888,  2.5070,  2.5613],
        [-9.7174,  0.5533,  5.0000,  ...,  0.4275,  0.4347,  0.4563]],
       dtype=torch.float64)


In [5]:
import torch
from torch import nn

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(729, 400)
        self.fc21 = nn.Linear(400, 5)  # mu layer
        self.fc22 = nn.Linear(400, 5)  # logvariance layer
        self.fc3 = nn.Linear(5, 400)
        self.fc4 = nn.Linear(400, 729)

    def encode(self, x):
        h1 = torch.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = torch.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 729))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar
    
# z = self.encoder(x)
#         return self.decoder(z), z

        
    
    


In [45]:
input_dim = 727+2 # 727 laser readings + 2 pos readings
latent_dim = 5
num_epochs = 100
batch_size = 500

# Convert numpy array to PyTorch tensor
data_tensor = torch.from_numpy(data)

# Create a TensorDataset from the tensor
dataset = TensorDataset(data_tensor)

# Create a DataLoader from the TensorDataset
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
lr=1e-12
num_episodes = num_epochs
initial_lr = lr
final_lr = lr*1e-2
n_updates = num_episodes*0.7
lr_gamma =  (final_lr / initial_lr)**(1 / n_updates) # gamma

# vae = VAE2(input_dim, latent_dim).to(device)

# Example of how to use this loss function:
vae = VAE().to(device)
optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)


def loss_function(recon_x, x, mu, logvar):
    BCE = nn.functional.binary_cross_entropy(recon_x, x.view(-1, 729), reduction='sum')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD
    

for epoch in range(num_epochs):
    vae.train()
    train_loss = 0
    batch_idx=0
    for (inputs, ) in dataloader:
        inputs = inputs.to(torch.float32)
        batch_idx+=1
        optimizer.zero_grad()
        recon_batch, mu, logvar = vae(inputs)
        loss = loss_function(recon_batch, inputs, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()

        if batch_idx%50==0:print(train_loss)

    # for (inputs,) in dataloader:
    #     inputs = inputs.to(torch.float32).to(device)
    #     outputs, _ = vae(inputs)
    #     printloss =  F.mse_loss(outputs, inputs, reduction='sum') # 
    #     loss = F.binary_cross_entropy(outputs, inputs, reduction='sum')
    #     optimizer.zero_grad()
    #     loss.backward()
    #     optimizer.step()

-3371242401.921875
-7287151369.921875
-11243574601.921875
-15207751169.921875
-19191050305.921875
-23198194297.921875
-4045917688.0
-8078692008.0
-12123004416.0
-16175966008.0
-20239872128.0
-24304238152.0
-4059578856.0
-8136874952.0
-12207082608.0
-16283093816.0
-20352916176.0
-24423438120.0
-4073549960.0
-8154821512.0
-12240129296.0
-16313426248.0
-20393533080.0
-24467595120.0
-4069311392.0
-8158788072.0
-12236709432.0
-16316644624.0
-20395565816.0
-24484623256.0
-4091926768.0
-8174516640.0
-12269229080.0
-16346465152.0
-20429471208.0
-24506536768.0
-4079634224.0
-8170534896.0
-12263820240.0
-16353351832.0
-20434109304.0
-24526055200.0
-4087842112.0
-8174003048.0
-12259017088.0
-16351266352.0
-20439743936.0
-24530808656.0
-4087944224.0
-8178897880.0
-12258142368.0
-16344911888.0
-20432952216.0
-24524340200.0
-4097703328.0
-8186538080.0
-12281203672.0
-16377205984.0
-20452947560.0
-24544915808.0
-4087309448.0
-8179724176.0
-12276719024.0
-16364069400.0
-20443349176.0
-24550593968.0
-4

KeyboardInterrupt: 